# Installs

In [2]:
%%writefile requirements.txt

radiant-mlhub==0.4.1
rasterio
shapely
accelerate==0.15.0
albumentations==1.3.0
timm==0.6.12
watermark

Overwriting requirements.txt


In [1]:
!pip install -q torch==1.13.1 torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

In [2]:
!pip install -q -r requirements.txt

# Specs

In [3]:
!nvidia-smi

Fri Mar 10 23:45:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.05    Driver Version: 510.73.05    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:05.0 Off |                    0 |
| N/A   31C    P0    51W / 400W |      0MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Imports

In [4]:
import os, gc
import copy
import math
import random
import getpass
import pandas as pd
import numpy as np
from time import time, sleep
from tqdm.notebook import tqdm
from collections import Counter, defaultdict
from scipy.ndimage import gaussian_filter

from glob import glob
import warnings

import shutil

import subprocess
from joblib import Parallel, delayed
import multiprocessing

import cv2
import PIL
from PIL import Image
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import f1_score, mean_squared_error

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset as TDataset, DataLoader
from torch.optim.lr_scheduler import LambdaLR, CosineAnnealingLR

from accelerate import Accelerator

import albumentations as A
import albumentations.augmentations.geometric.transforms as AG

import timm

In [6]:
%load_ext watermark
%watermark --iversions

cv2           : 4.7.0
torch         : 1.13.1+cu116
PIL           : 9.2.0
albumentations: 1.3.0
timm          : 0.6.12
matplotlib    : 3.6.1
pandas        : 1.5.0
numpy         : 1.23.4



# Envs

In [7]:
def disable_warnings(strict=False):
	warnings.simplefilter('ignore')
	if strict:
		logging.disable(logging.WARNING)

def seed_everything(seed=42):
	random.seed(seed)
	os.environ['PYTHONHASHSEED'] = str(seed)
	np.random.seed(seed)
	torch.manual_seed(seed)
	torch.cuda.manual_seed(seed)
	torch.backends.cudnn.deterministic = True
	torch.backends.cudnn.benchmark = False

In [8]:
SEED = 42

disable_warnings()
seed_everything(SEED)

# Data

In [9]:
# load files
data_path = './data/'

train = pd.read_csv(data_path + 'Train.csv')
test = pd.read_csv(data_path + 'Test.csv')
sample_submission = pd.read_csv(data_path + 'SampleSubmission.csv')

# Preview train
train.head()

,ImageId,Target
0,Id_jdqw9hlv6j.png,14.0
1,Id_6xtrolmuvc.png,18.0
2,Id_2m49sj3xd9.png,0.0
3,Id_9jwg5pcnn4.png,28.0
4,Id_vnm6e8n0p3.png,21.0


In [10]:
# Preview test
test.head()

,ImageId
0,Id_ohk78h9ld8.png
1,Id_eeyj2u4j7y.png
2,Id_wsd7vx2ifa.png
3,Id_6vfneamaoh.png
4,Id_9wil3575fv.png


In [11]:
# Preview sample submission
sample_submission.head()

,ImageId,Target
0,Id_ohk78h9ld8.png,0
1,Id_eeyj2u4j7y.png,0
2,Id_wsd7vx2ifa.png,0
3,Id_6vfneamaoh.png,0
4,Id_9wil3575fv.png,0


# Utils

In [12]:
def paralellize(fct, data, verbose=0, with_tqdm=True):
    fn = map(delayed(fct), data)
    if with_tqdm:
        fn = tqdm(fn, total=len(data))
    return Parallel(n_jobs=-1, verbose=verbose, backend="multiprocessing")(fn)

In [13]:
def get_common_transforms(size):
    return A.Compose([
      A.Resize(height=size, width=size, p=1),
    ])

def get_train_transforms():
    return A.Compose([
        A.Flip(),
        A.RandomRotate90(),
        A.OneOf([
            A.MotionBlur(),
            A.MedianBlur(blur_limit=3),
            A.Blur(blur_limit=3),
        ]),
    ])

def get_mosaic_transforms():
    return A.Compose([
        A.Flip(),
        A.RandomRotate90(),
        A.ShiftScaleRotate(
            shift_limit=0.0625, scale_limit=(-0.3, -0.2), rotate_limit=45, interpolation=1, border_mode=0, value=0, p=0.2
        )
    ])

In [14]:
def get_concat_h(im1, im2):
    dst = Image.new('RGB', (im1.width + im2.width, im1.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (im1.width, 0))
    return dst

def get_concat_v(im1, im2):
    dst = Image.new('RGB', (im1.width, im1.height + im2.height))
    dst.paste(im1, (0, 0))
    dst.paste(im2, (0, im1.height))
    return dst

def get_combos(ids, n=4) :
    combos = []
    size = len(ids)
    
    for i in range(0, size - n, 1):
        combos.append(ids[i:i+n])
    return combos

def _combo_image_saver(ids_list):
    save_path = './data/TreeImagesCombo/'
    
    images = [
        Image.open(f'./data/TreeImages/{p}') for p in ids_list
    ]

    img = get_concat_h(*images[:2])

    if len(ids_list) == 4:
        img = get_concat_v(
            img,
            get_concat_h(*images[2:])
        )

    ids_list = [p.split('.')[0] for p in ids_list]
    path = save_path + '|'.join(ids_list) + '.png'

    img.save(path)

def concat_mix_up(zeros_ids, palm_ids, n=4):
    np.random.shuffle(zeros_ids)
    np.random.shuffle(palm_ids)
    
    palm_combos = []
    if n != -1:
        palm_combos = get_combos(palm_ids, n=n)
    
    new_palm_combos = list(zip(zeros_ids, palm_ids))
    new_palm_combos = [list(x) for x in new_palm_combos]
    
    _ = paralellize(_combo_image_saver, palm_combos + new_palm_combos)
    
    print(f'New {len(palm_combos + new_palm_combos)} images generated')
    
    return palm_combos + new_palm_combos

In [15]:
def mosaic(imgs):
    size = len(imgs)
    h, w, c = imgs[0].shape
    
    assert 1 <= size <= 4, f'size is {size}'
    
    if size == 1:
        img = imgs[0]
    elif size == 2:
        img = np.empty(shape=(h, w*size, c), dtype=imgs[0].dtype)
        img[:, :w, :] = imgs[0]
        img[:, w:, :] = imgs[1]
    else:
        if size == 3:
            imgs.append(np.zeros_like(imgs[0], dtype=imgs[0].dtype))
        
        img = np.empty(shape=(h*2, w*2, c), dtype=imgs[0].dtype)
        img[:h, :w, :] = imgs[0]
        img[h:, :w, :] = imgs[1]
        img[:h, w:, :] = imgs[2]
        img[h:, w:, :] = imgs[3]
    
    return img

In [21]:
def criterion(preds, counting):
    loss_fn = nn.MSELoss()
    return torch.sqrt(loss_fn(preds, counting))

def metric(preds, biomass):
    preds = preds.cpu().numpy()
    preds = np.where(preds > 0, preds, 0)
    
    return mean_squared_error(biomass.cpu().numpy(), preds, squared=False)

In [22]:
class TreeCountingModel(nn.Module):
    def __init__(self, args):
        super().__init__()

        self.backbone = timm.create_model(
            args.arch, num_classes=1, pretrained=True, **args.extra_params
        )

    def forward(self, x):
        x = self.backbone(x)
        return x

In [23]:
class TreeCountingDataset(torch.utils.data.Dataset):
    def __init__(self, args, imagesPath, df, tfms=None, phase='test'):
        self.imagespath = imagesPath
        self.df = df.reset_index(drop=True)
        self.tfms = tfms
        self.default_tfms = get_common_transforms(args.img_size)
        self.mosaic = args.mosaic
        self.mosaic_aug = args.mosaic_aug
        self.mosaic_tfms = get_mosaic_transforms()
        self.length = len(self.df)
        self.phase = phase

    def __len__(self):
        return self.length
    
    def _read_image(self, idx):
        imagename, label, *z = self.df.iloc[idx].values
        path = os.path.join(self.imagespath, imagename)
        if not os.path.exists(path):
            path = os.path.join('./data/TreeImagesCombo/', imagename)
        image = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
        
        return image, label

    def __getitem__(self, idx):
        image, label = self._read_image(idx)
        
        if self.mosaic and self.phase == 'train':
            n_image = np.random.choice([0, 1])
            if n_image != 0:
                idxs = np.random.randint(self.length, size=n_image)
                images = [image]
                for i in idxs:
                    img, i_label = self._read_image(i)
                    if self.mosaic_aug:
                        img = self.mosaic_tfms(image=img)['image']
                        
                    images.append(img)
                    label += i_label
                
                image = mosaic(images)
        
        if self.tfms is not None:
            image = self.tfms(image=image)['image']
        
        image = self.default_tfms(image=image)['image']
        image = torch.tensor(image, dtype=torch.float)
        image = torch.permute(image, (2, 0, 1))
        
        return image, torch.tensor([label], dtype=torch.float)

In [24]:
def get_model(args):
    model = TreeCountingModel(args)

    return model

In [25]:
def make_splits(args, df, path='folds/'):
    os.makedirs("folds/", exist_ok=True)
    os.makedirs(f"folds/{args.n_split}_splits/", exist_ok=True)
    
    all_splits = {}
    fold_path = f"folds/{args.n_split}_splits/"
    
    df_clean = df.copy()
    
    to_drop = ['Id_u45dpub99b.png', 'Id_kzuy1er5jm.png', 'Id_lp4yl8q9n2.png', 'Id_2r2259ynzp.png', 'Id_w4fnd54go8.png', 'Id_lh8b1k1lx8.png', 'Id_hmcnf15rll.png']
    hard_ids = ['Id_scf1bqac43.png', 'Id_uhcehdmyc1.png', 'Id_y87obok3v0.png', 'Id_kx60b4dgn2.png', 'Id_85992p4se2.png', 'Id_g7fbxrng46.png',
           'Id_tz74cy4nh9.png', 'Id_u2dzh57wmf.png', 'Id_n0ipc9o8el.png', 'Id_lvua92vvg5.png', 'Id_ibrflyue8t.png', 'Id_xi61ipimpj.png',
           'Id_tn748l5k57.png', 'Id_63zqtghlbi.png', 'Id_9xwykwajjb.png', 'Id_owxhn1zmlh.png', 'Id_6ls94ewz47.png', 'Id_upua3g7q2q.png',
           'Id_eza4ic1a4k.png', 'Id_jxu8riswpw.png', 'Id_4nuo9gc80n.png', 'Id_rkm6vwm0md.png', 'Id_2goic5xoft.png', 'Id_234nhwag1a.png',
           'Id_842atkbl2g.png', 'Id_v9gmdly1tl.png', 'Id_7evqslin0q.png', 'Id_5jgknuixmg.png', 'Id_8bhp0orrri.png', 'Id_yyvlvcursz.png',
           'Id_kbgnlekbjm.png', 'Id_kd9m5eoeuf.png', 'Id_h0f091di0y.png', 'Id_scf4jxhlfg.png', 'Id_8xuqs2ut71.png', 'Id_4x0zh3y93q.png',
           'Id_oamy4tybj9.png', 'Id_8q73x3g38y.png', 'Id_k0c89tgg4c.png', 'Id_niwpztq309.png', 'Id_r405nptkvd.png'
    ]
    df_hard = df_clean[df_clean.ImageId.isin(hard_ids)].reset_index(drop=True)
    df_clean = df_clean[~df_clean.ImageId.isin(to_drop+hard_ids)].reset_index(drop=True)
    
    kf = StratifiedKFold(args.n_split, shuffle=True, random_state=42)
    
    num_bins = int(np.floor(1 + np.log2(len(df_clean))))
    bins = pd.cut(df_clean['Target'], bins=num_bins, labels=False)
        
    print('Num bins:', num_bins)
    
    for f, (tr_idx, vr_idx) in enumerate(kf.split(df_clean, bins)):
        all_splits[f] = {
          'train': pd.concat([ df_clean.iloc[tr_idx], df_hard]).sample(frac=1),
          'val': df_clean.iloc[vr_idx],
        }
        
        df_clean.iloc[vr_idx].to_csv(f"{fold_path}fold_{f}.csv", index=False)

    return all_splits

In [26]:
class Timer:
    def __init__(self):
        self._time = 0

    def start(self):
        self._time = time()

    @property
    def time(self):
        return (time() - self._time) / 60

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [27]:
def training_fn(args, dataloader, model, opt, epoch, accelerator, scheduler=None):    
    avg_loss = 0

    size = len(dataloader)
    timer = Timer()
    meter = AverageMeter()

    model.train()
    timer.start()
    for i, (x, counting) in enumerate(dataloader):
        preds = model(x)
        loss = criterion(preds, counting)

        avg_loss += loss.item()

        accelerator.backward(loss)
        opt.step()
        opt.zero_grad()

        score = metric(preds.detach().float(), counting.detach())
        meter.update(score)

        if scheduler is not None and args.schedule not in ['plateau']:
            scheduler.step()

        print(('\r[Training][{}/{}][{:.2f} min] Epoch {} : Loss: {:.5f} - RMSE: {:.5f}').format(
        i+1, size, timer.time, epoch, avg_loss/(i+1), meter.avg
        ), end='')
    print()

################## evaluation Function ####################   

def evaluate(dataloader, model):
    avg_loss = 0

    all_logits = []
    all_masks = []

    size = len(dataloader)
    timer = Timer()
    meter = AverageMeter()

    model.eval()
    timer.start()
    with torch.no_grad():
        for i, (x, counting) in enumerate(dataloader):
            preds = model(x)
            loss = criterion(preds, counting)

            avg_loss += loss.item()

            score = metric(preds.detach().float(), counting.detach().float())
            meter.update(score)

            print(('\r[Evaluation][{}/{}][{:.2f} min] Loss: {:.5f} - RMSE: {:.5f}').format(
            i+1, size, timer.time, avg_loss/(i+1), meter.avg
            ), end='')
    print()

    return avg_loss/size, meter.avg

################## Run training over folds Function ####################   
def release_memory():
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.reset_max_memory_allocated()
    torch.cuda.synchronize()

def run_fold(fold, args, path='./models'):
    model_path = f'{path}/{args.arch}-{args.n_split}-{args.img_size}-{args.lr}-{args.bs}'
    if fold == 0:
        mid = len(glob(model_path+'*'))
        model_path += f'-{mid}'
        os.makedirs(model_path, exist_ok=False)
        
        print('Model path:', model_path)
    else:
        model_id = sorted([int(x.replace(model_path+'-', '')) for x in glob(f'{model_path}*')])[-1]
        model_path = f'{model_path}-{model_id}'
    
    best_metric = np.inf
    best_loss = np.inf
   

    train_ds = TreeCountingDataset(
        args,
        imagesPath='./data/TreeImages/',
        df=all_splits[fold]['train'],
        tfms=get_train_transforms(),
        phase='train'
    )
    val_ds = TreeCountingDataset(
        args,
        imagesPath='./data/TreeImages/',
        df=all_splits[fold]['val'],
    )

    trainloader = DataLoader(train_ds, batch_size=args.bs, num_workers=args.workers, pin_memory=True, shuffle=True)
    validloader = DataLoader(val_ds, batch_size=args.ebs, num_workers=args.workers, pin_memory=True, shuffle=False)

    accelerator = Accelerator(fp16=args.fp16)

    model = get_model(args)
    opt = torch.optim.AdamW(model.parameters(), lr=args.lr, weight_decay=args.wd)

    num_train_steps = int(len(trainloader) * args.epochs)
    steps_per_epoch = len(trainloader)

    scheduler = None
    if args.schedule:
        scheduler = CosineAnnealingLR(opt, T_max=num_train_steps, eta_min=args.min_lr)
        model, scheduler, opt, trainloader, validloader = accelerator.prepare(model, scheduler, opt, trainloader, validloader)
    else:
        model, opt, trainloader, validloader = accelerator.prepare(model, opt, trainloader, validloader)

    os.makedirs(path, exist_ok=True)

    loader = tqdm(range(args.epochs), desc=f'Fold {fold}')

    for epoch in loader:
        training_fn(args, trainloader, model, opt, epoch, accelerator, scheduler)
        avg_loss, avg_metric = evaluate(validloader, model)

        if scheduler is not None and args.schedule in ['plateau']:
            scheduler.step(avg_loss)

        if avg_metric < best_metric:
            best_loss = avg_loss
            best_metric = avg_metric

            torch.save(model.state_dict(), f'{model_path}/best_{fold}.pt')
        print()


    print(f"Best Fold --- Loss score: {best_loss} - F1 Score {best_metric}")

    # Clean up
    del trainloader, validloader
    del model, opt
    release_memory()

    return best_metric

# Training

In [29]:
class CFG:
    arch = 'seresnet152d'
    lr = 5e-4
    min_lr = 0
    wd = 1e-6
    epochs = 20
    warmup = 0.
    bs = 24
    ebs = 2 * bs
    
    n_split = 5
    use_folds = list(range(n_split))
    actual_split = len(use_folds)

    img_size = 1024
    
    tta_tfms = []

    schedule = True
    augment = True
    mosaic = False
    mosaic_aug = False

    fp16 = True
    workers = os.cpu_count()
    
    print('Cpu count:', workers)

    extra_params = {
    }

Cpu count: 12


In [30]:
args = CFG()

In [31]:
all_splits = make_splits(args, train)

Num bins: 11


In [32]:
avg_score = 0

for f in args.use_folds:
    fold_score = run_fold(f, args)
    avg_score += (fold_score / args.actual_split)
    
    print()

Model path: ./models/seresnet152d-5-1024-0.0005-24-0


Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/seresnet152d_ra2-04464dd2.pth" to /root/.cache/torch/hub/checkpoints/seresnet152d_ra2-04464dd2.pth


Fold 0:   0%|          | 0/20 [00:00<?, ?it/s]

[Training][67/67][1.12 min] Epoch 0 : Loss: 5.73057 - RMSE: 5.7300472
[Evaluation][9/9][0.16 min] Loss: 2.30227 - RMSE: 2.30227

[Training][67/67][1.02 min] Epoch 1 : Loss: 2.68975 - RMSE: 2.68975
[Evaluation][9/9][0.12 min] Loss: 1.95413 - RMSE: 1.95412

[Training][67/67][1.01 min] Epoch 2 : Loss: 2.53179 - RMSE: 2.53178
[Evaluation][9/9][0.12 min] Loss: 2.07657 - RMSE: 2.07651

[Training][67/67][1.00 min] Epoch 3 : Loss: 2.28136 - RMSE: 2.28134
[Evaluation][9/9][0.12 min] Loss: 2.00166 - RMSE: 2.00165

[Training][67/67][1.00 min] Epoch 4 : Loss: 2.18072 - RMSE: 2.18060
[Evaluation][9/9][0.12 min] Loss: 1.87095 - RMSE: 1.87081

[Training][67/67][1.02 min] Epoch 5 : Loss: 1.89850 - RMSE: 1.89815
[Evaluation][9/9][0.13 min] Loss: 1.74113 - RMSE: 1.74044

[Training][67/67][1.01 min] Epoch 6 : Loss: 1.87604 - RMSE: 1.87584
[Evaluation][9/9][0.13 min] Loss: 1.66497 - RMSE: 1.66460

[Training][67/67][1.01 min] Epoch 7 : Loss: 1.76711 - RMSE: 1.76686
[Evaluation][9/9][0.13 min] Loss: 2.05438

Fold 1:   0%|          | 0/20 [00:00<?, ?it/s]

...

[Training][67/67][1.01 min] Epoch 0 : Loss: 5.70239 - RMSE: 5.7023987
[Evaluation][9/9][0.12 min] Loss: 2.32562 - RMSE: 2.32562

[Training][67/67][1.01 min] Epoch 1 : Loss: 2.90170 - RMSE: 2.90170
[Evaluation][9/9][0.12 min] Loss: 2.17924 - RMSE: 2.17924

[Training][67/67][1.00 min] Epoch 2 : Loss: 2.63901 - RMSE: 2.63901
[Evaluation][9/9][0.11 min] Loss: 3.46196 - RMSE: 3.46196

[Training][67/67][1.00 min] Epoch 3 : Loss: 2.33036 - RMSE: 2.33036
[Evaluation][9/9][0.12 min] Loss: 1.49258 - RMSE: 1.49258

[Training][67/67][1.02 min] Epoch 4 : Loss: 2.14495 - RMSE: 2.14493
[Evaluation][9/9][0.11 min] Loss: 2.12554 - RMSE: 2.12551

[Training][67/67][1.02 min] Epoch 5 : Loss: 1.91154 - RMSE: 1.91149
[Evaluation][9/9][0.13 min] Loss: 1.48348 - RMSE: 1.48345

[Training][67/67][1.01 min] Epoch 6 : Loss: 1.88618 - RMSE: 1.88614
[Evaluation][9/9][0.13 min] Loss: 1.38670 - RMSE: 1.38660

[Training][67/67][1.02 min] Epoch 7 : Loss: 1.67409 - RMSE: 1.67390
[Evaluation][9/9][0.13 min] Loss: 1.51514

Fold 3:   0%|          | 0/20 [00:00<?, ?it/s]

[Training][67/67][1.02 min] Epoch 0 : Loss: 5.84280 - RMSE: 5.8426731
[Evaluation][9/9][0.12 min] Loss: 2.87386 - RMSE: 2.87386

[Training][67/67][1.01 min] Epoch 1 : Loss: 3.10348 - RMSE: 3.10348
[Evaluation][9/9][0.12 min] Loss: 1.69054 - RMSE: 1.69054

[Training][67/67][1.02 min] Epoch 2 : Loss: 2.40421 - RMSE: 2.40419
[Evaluation][9/9][0.13 min] Loss: 1.67365 - RMSE: 1.67365

[Training][67/67][1.02 min] Epoch 3 : Loss: 2.39659 - RMSE: 2.39656
[Evaluation][9/9][0.13 min] Loss: 1.67212 - RMSE: 1.67191

[Training][67/67][1.01 min] Epoch 4 : Loss: 1.98476 - RMSE: 1.98463
[Evaluation][9/9][0.13 min] Loss: 1.72004 - RMSE: 1.71977

[Training][67/67][1.02 min] Epoch 5 : Loss: 2.06976 - RMSE: 2.06964
[Evaluation][9/9][0.12 min] Loss: 1.74030 - RMSE: 1.73988

[Training][67/67][1.01 min] Epoch 6 : Loss: 1.92516 - RMSE: 1.92494
[Evaluation][9/9][0.13 min] Loss: 1.60464 - RMSE: 1.60418

[Training][67/67][1.02 min] Epoch 7 : Loss: 1.89011 - RMSE: 1.88976
[Evaluation][9/9][0.12 min] Loss: 1.81764

Fold 4:   0%|          | 0/20 [00:00<?, ?it/s]

[Training][67/67][1.02 min] Epoch 0 : Loss: 5.38365 - RMSE: 5.3836501
[Evaluation][9/9][0.13 min] Loss: 2.48500 - RMSE: 2.48500

[Training][67/67][1.02 min] Epoch 1 : Loss: 3.15165 - RMSE: 3.15165
[Evaluation][9/9][0.13 min] Loss: 1.79706 - RMSE: 1.79706

[Training][67/67][1.01 min] Epoch 2 : Loss: 2.35033 - RMSE: 2.35029
[Evaluation][9/9][0.12 min] Loss: 1.76246 - RMSE: 1.76190

[Training][67/67][1.01 min] Epoch 3 : Loss: 2.15078 - RMSE: 2.15069
[Evaluation][9/9][0.12 min] Loss: 1.91593 - RMSE: 1.91585

[Training][67/67][1.01 min] Epoch 4 : Loss: 2.28176 - RMSE: 2.28123
[Evaluation][9/9][0.13 min] Loss: 2.02925 - RMSE: 2.02766

[Training][67/67][1.01 min] Epoch 5 : Loss: 1.96411 - RMSE: 1.96344
[Evaluation][9/9][0.12 min] Loss: 1.43031 - RMSE: 1.42997

[Training][67/67][1.03 min] Epoch 6 : Loss: 1.87776 - RMSE: 1.87688
[Evaluation][9/9][0.13 min] Loss: 1.68136 - RMSE: 1.68070

[Training][67/67][1.02 min] Epoch 7 : Loss: 1.85537 - RMSE: 1.85472
[Evaluation][9/9][0.13 min] Loss: 1.65186

In [33]:
avg_score

1.3432142164972094